# 📘 发票信息提取模型常见训练错误全解析（Donut / LayoutLMv3 / OCR Prompt）

## 🎯 模型目标输出样式（结构化 JSON）

```json
{
  "InvoiceNo": "Y 309824263008",
  "InvoiceDate": "2025年6月30日",
  "Currency": "USD",
  "Amount with Tax": "300",
  "Amount without Tax": "300",
  "Tax": "0"
}

### 🔶 模型一：Donut（无 OCR、多模态 Encoder-Decoder）

#### ✅ 关键点

- 输入：图像 → pixel_values（Tensor）
- 输出：结构化 JSON 序列
- 无需 OCR、word box 坐标

#### ❌ 常见错误与解决方案（训练阶段）

| 错误信息                                      | 错误类型     | 可能原因                                | 解决方案                                                                 |
|---------------------------------------------|------------|-----------------------------------------|--------------------------------------------------------------------------|
| `decoder_start_token_id not set`            | 配置缺失     | 未设置生成起始 token                    | 加：`model.config.decoder_start_token_id = processor.tokenizer.convert_tokens_to_ids(["<s>"])[0]` |
| `pad_token_id missing`                      | 配置缺失     | 未设置 padding token                    | 加：`model.config.pad_token_id = processor.tokenizer.pad_token_id`      |
| `pixel_values 应为 Tensor，而不是 list`     | 数据预处理   | 图像没 `squeeze(0)` 或 `map()` 未生效   | 在 `processor(...).pixel_values.squeeze(0)` 后返回；或在 `with_transform` 显式转 Tensor |
| `tokenizer is deprecated`                   | 接口警告     | HuggingFace API 变动                   | 推荐替换：`tokenizer= → processor=processor`（`transformers>=4.40`）   |
| `input_ids not in input`                    | collator问题 | 缺字段或结构错误                        | 检查 `collator` 返回是否包含 `pixel_values` 和 `labels`                 |
| `Unable to avoid copy while creating np.array` | NumPy 2.0   | `datasets` 转换张量时触发              | 将 `np.array(..., copy=False)` 改为 `np.asarray(...)`；或降级 numpy 到 `<2.0` |

| 错误类型                                 | 潜在原因                                                   | 排查与解决方案                                                                 |
|------------------------------------------|------------------------------------------------------------|--------------------------------------------------------------------------------|
| CUDA out of memory 或资源不足            | 图像太大、batch size 过大、模型太大                        | 调小图像尺寸（如 thumbnail 调成 960x720），或使用更小模型如 donut-base       |
| RuntimeError: Expected all tensors to be on the same device | 某些 Tensor 没有 `.to(device)`                            | 检查 `decoder_input_ids`、`pixel_values` 是否都加了 `.to(device)`              |
| OSError: Can't load tokenizer            | 依赖项版本不兼容或 Transformers 本地模型加载失败           | 升级 `transformers` 和 `Pillow`，并尝试重新下载模型（清除缓存）              |
| tokenizer.decode 出现乱码或无结果       | 模型输出乱码或没加 `skip_special_tokens=True`             | 保持 `skip_special_tokens=True`，并检查输入 prompt 格式是否规范              |
| 图像打不开或损坏                         | 非 RGB 图像、路径错误、压缩图片太小                        | 使用 `.convert("RGB")`，并加入图片尺寸判断 `image.size`                      |
| generate() 报错或结果为空                | prompt 格式不对、未加 `<s_docvqa>` 或 `<s_answer>` Token   | 使用格式：`"<s_docvqa>{question}<s_answer>"`                                 |

#### ❌ 推理阶段常见错误

| 错误信息                        | 原因                                      | 解决方案                                                                 |
|-------------------------------|-------------------------------------------|--------------------------------------------------------------------------|
| `json.decoder.JSONDecodeError` | 输出非 JSON，缺 `}` 或 `"` 等              | 使用 `try...except` 包裹 JSON 解析，结合正则修复：如添加结尾括号、补全引号等 |
| 输出全为空                      | 输入图像未标准化 / 模型未 fine-tune       | 确保图像 `.convert("RGB")` 并设置 `dpi >= 200`；或检查是否正确加载微调后的 checkpoint |

### 【错误一：CPU运行报错】

❌ 推論錯誤: xxxx.png, 問題: What is the invoice currency?, 原因: index out of range in self

Currency: 

AmountWithTax: 10

AmountWithoutTax: tax

Tax: 14:05


```
torch.nn.modules.module.ModuleAttributeError: 'VisionEncoderDecoderModel' object has no attribute 'base_model_prefix'

```

#### ❓原因分析：

该错误发生在调用 model.generate() 时内部处理模型属性时出错。DonutProcessor 是为 DonutModel 设计的，而你加载的是 VisionEncoderDecoderModel，两者不完全兼容。

虽然 Hugging Face 让你可以用 DonutProcessor 配合 VisionEncoderDecoderModel，但要确保模型是正确支持的结构。此处出现 base_model_prefix 报错，通常是因为 generate() 在 encoder-decoder 框架中找不到必要的属性。


#### 解决方案：

#### Before

In [ ]:
# 載入模型與處理器
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa").to(device)

### After

你应当使用 DonutModel 而不是 VisionEncoderDecoderModel：


In [ ]:
from transformers import DonutProcessor, DonutModel

processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")
model = DonutModel.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa").to(device) # <-- Fix Here

并注意 generate() 的参数也略有不同，推荐改写如下：

#### Before

In [ ]:
        with torch.no_grad():
            pixel_values = processor(image, return_tensors="pt").pixel_values.to(device)
            decoder_input_ids = processor.tokenizer(
                f"<s_docvqa>{question}<s_answer>",
                add_special_tokens=False,
                return_tensors="pt"
            ).input_ids.to(device)

            outputs = model.generate(pixel_values, decoder_input_ids=decoder_input_ids, max_length=512) # <-- Fix Here
            raw_answer = processor.tokenizer.decode(outputs[0], skip_special_tokens=True)
            return extract_answer(raw_answer)

#### After

In [ ]:
task_prompt = f"<s_docvqa>{question}</s_docvqa>"
decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids
outputs = model.generate(pixel_values, decoder_input_ids=decoder_input_ids.to(device), max_length=512)

### 【错误二：GPU运行时报错】

AmountWithoutTax: 

❌ 推論錯誤: xxxx.png, 問題: What is the tax amount?, 原因: CUDA error: device-side assert triggered

CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1

Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

它会 强制每一个 CUDA 操作同步执行，即：

- CUDA 操作（比如张量拷贝、模型计算）必须立即执行完毕，而不是异步执行；
- 这样能让你在出错时，准确地知道是哪一行代码出了问题。

```
RuntimeError: expected scalar type Half but found Float
```

#### ❓原因分析：

这是一个常见的类型不一致问题。在 GPU 模式下，某些模型（特别是被混合精度训练过的）会要求 tensor 类型是 torch.float16（也叫 Half），但你传的是 float32。

Half 是一种 标量数据类型（scalar type），表示 16 位浮点数（float16），常用于 GPU 上进行低精度加速计算，尤其是在深度学习中。

##### 📌 torch.HalfTensor 或 dtype=torch.float16 在 PyTorch 中的应用：

In [ ]:
import torch

# 创建 float16 张量
x = torch.tensor([1.0, 2.0, 3.0], dtype=torch.float16)
print(x.dtype)  # torch.float16

| 类型名称          | 位数  | 精度等级   | 举例             |
|-------------------|-------|------------|------------------|
| float / float32   | 32位  | 单精度     | 常用默认类型      |
| double / float64  | 64位  | 双精度     | 高精度计算用      |
| half / float16    | 16位  | 半精度     | 加速模型推理      |
| bfloat16          | 16位  | 特殊半精度 | 用于 TPU 上       |

#### 解决方案

你可以让模型和输入都使用 float16：

In [ ]:
model = model.half()  # 将模型转为 float16

同时确保图像 **tensor** 也转成 float16

In [ ]:
pixel_values = processor(image, return_tensors="pt").pixel_values.to(device).half()

### 🔶 模型二：LayoutLMv3（OCR + Layout NER 模型）

#### ✅ 关键点

- 输入包括：
- words：OCR token
- boxes：坐标（0~1000缩放）
- labels：BIO标签
- 适合用于位置感知抽取（如金额、发票号）

#### ❌ 常见训练错误与解决方案

| 错误信息                             | 错误类型     | 可能原因                          | 解决方案                                                                 |
|--------------------------------------|--------------|-----------------------------------|--------------------------------------------------------------------------|
| `words or boxes not found`           | 数据不合规   | Dataset 中缺字段                  | 确保每条样本都包含 `words`、`boxes`、`labels` 字段                        |
| `AssertionError: len(words) != len(boxes)` | OCR错位     | OCR 提取的文本顺序与 box 不一致   | 使用 PaddleOCR 的结构化输出或使用统一 OCR 工具保证顺序一致              |
| `Model training stuck`               | 无梯度变化   | 所有标签都是 O                    | 确保部分样本中有非 O 标签，如 `B-InvoiceNo` 等                           |
| `CUDA out of memory`                 | 显存不足     | batch_size 太大或图像分辨率太高  | 减小 `batch_size`，限制图像大小为 `224x224` 或 `384x384`                |

#### ❌ 推理问题与建议

| 问题                     | 原因                         | 建议                                                                 |
|--------------------------|------------------------------|----------------------------------------------------------------------|
| 发票字段提取不全         | 标签缺失 / 标注不一致        | 使用 [Label Studio](https://labelstud.io/) 标注实体范围，保持格式统一 |
| 英文字段准确率高、中文差 | 模型预训练语料不均            | 使用中英文混合数据进行微调，或选用 `layoutlmv3-chinese-base` 等模型  |

### 🔶 模型三：OCR + 正则 / Prompt 模型（MiniCPM, ChatGLM等）

#### ✅ 使用场景
- 无 GPU 或只需少量样本
- 发票样式可归类、具语义结构
- 与大模型结合，适合 prompt-based 抽取

#### ❌ OCR常见问题

### 🧩 发票识别常见问题分析（三）

| 问题         | 原因                     | 解决方案                                                                 |
|--------------|--------------------------|--------------------------------------------------------------------------|
| 字体识别错位 | DPI 过低 / 图像噪声     | 转图像时设置 `dpi=200~300`，建议使用 `pdf2image` 替代 `fitz` 提高清晰度 |
| 字段分行错   | 多栏位内容混在一起       | 使用 `structure=True`（如 PaddleOCR）输出多段块，并按 `bbox` 排序        |
| 日期字段错   | “年 月 日” 结构变化大   | 用正则表达式提取，例如：`r"\d{4}年\d{1,2}月\d{1,2}日"` 或规则模板匹配     |

#### ❌ 正则 + Prompt 模型错误

| 错误类型         | 原因             | 解决方法                                 |
|------------------|------------------|------------------------------------------|
| 正则无法匹配     | 格式差异大       | 用 re.findall() 加多个模板方案（模糊匹配） |
| Prompt 模型不稳定 | 输入换行/空格混乱 | 使用 textwrap.dedent 预处理文本           |
| 输出非结构化 JSON | 模型自由回答不受控 | 加入强制约束 Prompt：如 “请用 JSON 格式回答以下问题…” |